In [1]:
# # from functools import partial
from pathlib import Path
# from typing import Optional, Tuple
# import cv2
# import fire
import numpy as np
import torch
import torch.nn.functional as F
from accelerate import Accelerator
from PIL import Image
from scipy.sparse.linalg import eigsh
# from sklearn.cluster import KMeans, MiniBatchKMeans
# from sklearn.decomposition import PCA
# from torchvision.utils import draw_bounding_boxes
from tqdm import tqdm
import extract_utils as utils
from torch.utils.data import Dataset, DataLoader

import torchvision
from torch import nn
from argparse import Namespace

## Importing Features

In [2]:
features_dir="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/features/dino_vits16"
inputs = list(enumerate(sorted(Path(features_dir).iterdir())))
which_features='k'

In [3]:
# print("type of inputs=", type(inputs))
print("len of inputs=",len(inputs))
print("type of inputs[0]", type(inputs[0]))
print("length of inputs[0]=",len(inputs[0]))
print("inputs[0][0]=",inputs[0][0])
print("inputs[0][1]=", inputs[0][1])

len of inputs= 1000
type of inputs[0] <class 'tuple'>
length of inputs[0]= 2
inputs[0][0]= 0
inputs[0][1]= /home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/features/dino_vits16/0001.pth


In [4]:
feat_list=[]
for inp in tqdm(inputs[:1]):
    index, features_file = inp
    # print(index, features_file)
     # Load
    data_dict = torch.load(features_file, map_location='cpu')
    feats = data_dict[which_features].squeeze().cuda()
    print(feats.shape)
    # tensor_features=torch.stack(torch.tensor(feats))
    feat_list.append(feats)
    # print("data_dict.keys=", data_dict.keys())
    # print('data_dict[k]:', data_dict['k'])
    # print('data_dict[k].shape=', data_dict['k'].shape)
    # print('data_dict[indices]:', data_dict['indices'])
    # print('data_dict[file]:', data_dict['file'])
    # print('data_dict[model_name]:', data_dict['model_name'])
    # print('data_dict[patch_size]:', data_dict['patch_size'])
    # print('data_dict[shape]:', data_dict['shape'])
    # print('type of data_dict[shape].shape=', type(data_dict['shape']))
    # print('data_dict[shape][0]:', data_dict['shape'][0])
    # print('data_dict[shape][1]:', data_dict['shape'][1])
    # print('data_dict[shape][2]:', data_dict['shape'][2])
    # print('data_dict[shape][3]:', data_dict['shape'][3])


100%|██████████| 1/1 [00:00<00:00,  1.83it/s]

torch.Size([400, 384])


In [5]:
training_args={
    'arch':"resnet50",
    'dim':2048,
    'pred_dim':512,
    'fix_pred_lr':'store_true',
    'momentum':0.9,
    'wd': 1e-4,
    'lr':0.05,
    'batch_size':8,
    'data': "/home/phdcs2/Hard_Disk/Projects/Codes/lightly/examples/pytorch/datasets/cifar10",
    'workers':32,
    'start_epoch':0,
    'epochs':10
}
args = Namespace(**training_args)

## Custom Dataset on FeatureSpace

In [6]:
print(len(feat_list))
print(type(feat_list[0]))
print(feat_list[0].shape)

1
<class 'torch.Tensor'>
torch.Size([400, 384])


In [7]:
# Define a custom dataset class
class Feature_Dataset(Dataset):
    def __init__(self, features):
        self.features = features

    def __getitem__(self, index):
        return self.features[index]

    def __len__(self):
        return len(self.features)

In [8]:
# features=data_dict['k']
# print(features.shape)
feat_dataset = Feature_Dataset(feat_list[:1])
features_dataloader = DataLoader(feat_dataset, batch_size=1, shuffle=True)

In [9]:
class SimSiam(nn.Module):
    def __init__(self):
        super().__init__()
        self.dim=128
        self.pred_dim=feats.shape[1]
        self.projection_head = nn.Sequential(nn.Linear(feats.shape[1], 128, bias=False),
                                        nn.BatchNorm1d(128),
                                        nn.ReLU(inplace=True), # first layer
                                        nn.Linear(128, self.pred_dim, bias=False),
                                        nn.BatchNorm1d(self.pred_dim),
                                        nn.ReLU(inplace=True), # second layer
                                        # self.encoder.fc,
                                        nn.BatchNorm1d(self.pred_dim, affine=False)) # output layer
        self.prediction_head = nn.Sequential(nn.Linear(self.pred_dim, self.dim, bias=False),
                                        nn.BatchNorm1d(self.dim),
                                        nn.ReLU(inplace=True), # hidden layer
                                        nn.Linear(self.dim, self.pred_dim,)) # output layer

    def forward(self, x1, x2):
        """
        Input:
            x1: first views of images
            x2: second views of images
        Output:
            p1, p2, z1, z2: predictors and targets of the network
            See Sec. 3 of https://arxiv.org/abs/2011.10566 for detailed notations
        """

        # compute features for one view
        z1 = self.projection_head(x1) # NxC
        z2 = self.projection_head(x2) # NxC

        p1 = self.prediction_head(z1) # NxC
        p2 = self.prediction_head(z2) # NxC

        return p1, p2, z1.detach(), z2.detach()

In [10]:
model_simsiam = SimSiam()
device = "cuda" if torch.cuda.is_available() else "cpu"
model_simsiam.to(device)

SimSiam(
  (projection_head): Sequential(
    (0): Linear(in_features=384, out_features=128, bias=False)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Linear(in_features=128, out_features=384, bias=False)
    (4): BatchNorm1d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): BatchNorm1d(384, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  )
  (prediction_head): Sequential(
    (0): Linear(in_features=384, out_features=128, bias=False)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Linear(in_features=128, out_features=384, bias=True)
  )
)

In [11]:
criterion = nn.CosineSimilarity(dim=1).cuda(0)
optimizer = torch.optim.SGD(model_simsiam.parameters(), lr=0.06,momentum=args.momentum,
                                weight_decay=args.wd)

In [12]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [13]:
# print("Starting Training")
losses = AverageMeter('Loss', ':.4f')
for epoch in range(10):
    total_loss = 0
    for x0 in features_dataloader:
    # for (x0), _, _ in features_dataloader:
        x1=torchvision.transforms.RandomAffine(0)(x0)
        x0 = x0.squeeze().to(device)
        # x0 = x0.to(device)
        # print("x0.shape=", x0.shape)
        x1 = x1.squeeze().to(device)
        # print("x1 shape=", x1.shape)
        p1, p2, z1, z2 = model_simsiam(x1=x0, x2=x1)
        print("p1.shape=", p1.shape)
        print("p2.shape=", p2.shape)
        print("z1.shape=", z1.shape)
        print("z2.shape=", z2.shape)
        loss = -(criterion(p1, z2).mean() + criterion(p2, z1).mean()) * 0.5
        total_loss += loss.detach()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # losses.update(loss.item(), x0.size(0))
        #
        # # compute gradient and do SGD step
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()
    avg_loss = total_loss / len(features_dataloader)
    print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")

p1.shape= torch.Size([400, 384])
p2.shape= torch.Size([400, 384])
z1.shape= torch.Size([400, 384])
z2.shape= torch.Size([400, 384])
epoch: 00, loss: 0.00579
p1.shape= torch.Size([400, 384])
p2.shape= torch.Size([400, 384])
z1.shape= torch.Size([400, 384])
z2.shape= torch.Size([400, 384])
epoch: 01, loss: -0.01513
p1.shape= torch.Size([400, 384])
p2.shape= torch.Size([400, 384])
z1.shape= torch.Size([400, 384])
z2.shape= torch.Size([400, 384])
epoch: 02, loss: -0.04892
p1.shape= torch.Size([400, 384])
p2.shape= torch.Size([400, 384])
z1.shape= torch.Size([400, 384])
z2.shape= torch.Size([400, 384])
epoch: 03, loss: -0.08325
p1.shape= torch.Size([400, 384])
p2.shape= torch.Size([400, 384])
z1.shape= torch.Size([400, 384])
z2.shape= torch.Size([400, 384])
epoch: 04, loss: -0.11363
p1.shape= torch.Size([400, 384])
p2.shape= torch.Size([400, 384])
z1.shape= torch.Size([400, 384])
z2.shape= torch.Size([400, 384])
epoch: 05, loss: -0.13930
p1.shape= torch.Size([400, 384])
p2.shape= torch.Size